In [1]:
import csv
import os
import sys
import random
import string
import pandas as pd
import pyspark
from sklearn.utils import shuffle
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import RegexpTokenizer
from pyspark.sql import SQLContext
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, size, max, abs

In [5]:
# Initialize a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
# Label the data sets and remove unncessary fields
fake = pd.read_csv("../data/Fake.csv")
true = pd.read_csv("../data/True.csv")
fake = fake.assign(label="Fake")
fake = fake.drop(columns=["date", "subject"])
true = true.assign(label="True")
true = true.drop(columns=["date", "subject"])
fake = fake.head(2150)
true = true.head(2150)
fake.to_csv("../data/Sample_Fake.csv", index=False, header=None)
true.to_csv("../data/Sample_True.csv", index=False, header=None)

In [10]:
# Removing stop words from the article context 
punc = list(string.punctuation) + ['“','”','‘', '’','...']
stop_words = stopwords.words('english') + punc
true_rdd = spark.read.csv("../data/Sample_True.csv").rdd
true_rdd = true_rdd.map(lambda x: [x[0], x[1], x[2]])
true_rdd = true_rdd.map(lambda x: [x[0], [i for i in word_tokenize(x[1].lower()) if i not in stop_words], x[2]])
fake_rdd = spark.read.csv("../data/Sample_Fake.csv").rdd
fake_rdd = fake_rdd.map(lambda x: [x[0], x[1], x[2]]).filter(lambda x: x[1] is not None)
fake_rdd = fake_rdd.map(lambda x: [x[0], [i for i in word_tokenize(x[1].lower()) if i not in stop_words], x[2]])